# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

.env file created!


In [ ]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv("chain.env"))

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

print("API keys loaded.")


API keys loaded.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
!pip install pandas

In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
from google.colab import files
files.upload()


Saving Data.csv to Data.csv


{'Data.csv': b'Product,Review\nQueen Size Sheet Set,"I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets."\nWaterproof Phone Pouch,"I loved the waterproof sac, although the opening was made of a hard plastic. I don\xe2\x80\x99t know if that would break easily. But I couldn\xe2\x80\x99t turn my phone on, once it was in the pouch."\nLuxury Air Mattress,"This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it\'s the top of the mattress where it\'s kind of like fabr

In [ ]:
import os
print(os.listdir('/content'))


['.config', 'chain.env', 'Data.csv', 'sample_data']


In [ ]:
import pandas as pd
df = pd.read_csv("Data.csv")



In [ ]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv('/content/chain.env')


OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

print("API keys loaded successfully.")


API keys loaded successfully.


## LLMChain

In [ ]:
!pip install langchain_community
!pip install langchain
!pip install openai
!pip install langchain



In [ ]:
pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
#Replace None by your own value and justify

llm = ChatOpenAI(temperature=0.7)


In [ ]:
prompt = ChatPromptTemplate.from_template("Describe the product: {product}")


In [ ]:

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = df['Product'][0]
result = prompt | llm
description = result.invoke({"product": product})
print(description)


## SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# Step 2: Define the first prompt template
first_prompt = ChatPromptTemplate.from_template(
    "Describe the product in detail: {product}"
)

# Chain 1: Create LLMChain with the first prompt
chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [ ]:

# Step 3: Define the second prompt template
second_prompt = ChatPromptTemplate.from_template(
    "Write a catchy ad for this product: {input}"
)

# Chain 2: Create LLMChain with the second prompt
chain_two = LLMChain(llm=llm, prompt=second_prompt)


In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)

**Repeat the above twice for different products**

In [ ]:
# Use SimpleSequentialChain with two different products
products = [df['Product'][0], df['Product'][1]]  # Select two products from the data

for product in products:
    # Execute the chain on each product
    result = overall_simple_chain.invoke({"product": product})
    print(f"Result for product '{product}':")
    print(result)
    print("\n" + "="*50 + "\n")


## SequentialChain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
# Initialize the OpenAI model with temperature 0.9
llm = ChatOpenAI(temperature=0.9)

# Set up the first prompt to translate the review
first_prompt = ChatPromptTemplate.from_template(
  "Translate this review to English:\n{Review}"
)

# Create the first chain (chain_one)
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="translated_review"  # Specify the output key
                    )


In [ ]:
# Set up the second prompt to summarize the review
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review:\n{translated_review}"
)

# Create the second chain (chain_two)
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"  # Specify the output key
                    )


In [ ]:
# Third prompt template: translate to English or another language
third_prompt = ChatPromptTemplate.from_template(
    "Translate this review to English:\n{Review}"  # You can customize the translation to another language if needed
)

# Create the third chain (chain_three): input = Review, output = translated review or language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_review"  # Output key will be translated_review
                      )


In [ ]:
# prompt template 4: follow-up message that takes as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "The review was translated to English as: {translated_review}. "
    "Here is the summary: {summary}. "
    "Write a polite thank-you message to the reviewer, addressing any concerns they might have."
)

# chain 4: input = translated_review, summary, output = follow-up message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"  # اسم المخرج سيكون followup_message
                     )


In [ ]:
# chain_one: Translate the review (output_key should be translated_review)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="translated_review")

# chain_two: Summarize the review (output_key should be summary)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

# chain_three: Detect the language (output_key should be language)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

# chain_four: Follow-up message (output_key should be followup_message)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

# overall_chain: input = Review
# and output = translated_review, summary, language, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],  # This is the variable that will take the original review as input
    output_variables=["translated_review", "summary", "language", "followup_message"],  # These are the results we expect
    verbose=True  # This will display the detailed output during execution
)


In [ ]:
review = df.Review[5]
overall_chain(review)

**Repeat the above twice for different products or reviews**

In [ ]:
# Define the first product and second product
product_1 = df['Product'][0]  # First product
product_2 = df['Product'][1]  # Second product

# Execute SimpleSequentialChain or SequentialChain for the first product
description_1 = overall_simple_chain.run(product_1)
print(f"Product 1 Description: {description_1}")

# Execute SimpleSequentialChain or SequentialChain for the second product
description_2 = overall_simple_chain.run(product_2)
print(f"Product 2 Description: {description_2}")


In [ ]:
# Define the first review and second review
review_1 = df['Review'][0]  # First review
review_2 = df['Review'][1]  # Second review

# Execute SequentialChain for the first review
result_1 = overall_chain.invoke({"Review": review_1})
print(f"Review 1 Processed: {result_1}")

# Execute SequentialChain for the second review
result_2 = overall_chain.invoke({"Review": review_2})
print(f"Review 2 Processed: {result_2}")


## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(temperature=0)

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# Create destination descriptions string
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)


In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}


In [ ]:

#### Router Chain**:

from langchain.chains.router import LLMRouterChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import RouterOutputParser

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)


In [ ]:
##MultiPromptChain
from langchain.chains.router import MultiPromptChain

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain, verbose=True
)


In [ ]:
result_1 = chain.run("What is Schrödinger's cat paradox?")
print(result_1)


In [ ]:
result_2 = chain.run("What is the Fibonacci sequence and how is it applied?")
print(result_2)


In [ ]:
result_3 = chain.run("Who was Julius Caesar and why is he important?")
print(result_3)
